In [1]:
import numpy as np
import cv2
import math
#import skvideo.io
def transformImage(image, src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)
    return warped

def show_image(image):
    key = 0;
    while (key != 27):
        cv2.imshow("Image", image)
        key = cv2.waitKey(10)
    cv2.destroyAllWindows()

In [35]:
def center_path(img, img_eroded, min_y, max_y, direction, center_x_old, tolerance, sum_required, straight_count, straight_flag, count, junction_flag, junction_frame):
    global center_x
    center_x = 320
    ### detect center of lanes
    histogram = []
    point = []
    point_1=[]
    point_2=[]
    sum_f = []
    width=128
    text_scale=0.3
    color = (255,255,255)
    thickness = 1
    
    histogram = np.zeros((width,), dtype=int)
#    img_roi = img_eroded[min_y:max_y, :, 1] #if 3 channels
    img_roi = img_eroded[min_y:max_y, :]
    histogram = np.apply_along_axis(np.count_nonzero, 0, img_roi)
    
    for w in range(0, len(histogram)-1):
        if (histogram[w]!=0 and histogram[w-1]==0) or (histogram[w]!=0 and histogram[w+1]==0):
            point.append(w)
            
    for no in range(0,len(point)-1):
        if np.sum(histogram[point[no]:point[no+1]]) > sum_required :
            point_1.append(point[no])
            point_2.append(point[no+1])
    #print histogram        
    ### detect center of path
    if len(point_1) == 1:
        cv2.putText(img,'1 lane detected',(0,120),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        center_lanes = (point_1[0]+point_2[0])/2
        if junction_flag == 1:
            if direction == "left":
                if center_lanes<43:
                    center_x = (center_lanes+128)/2
                else:
                    center_x = center_lanes/2
            elif direction == "right":
                if center_lanes>86:
                    center_x = center_lanes/2
                else:
                    center_x = (center_lanes+128)/2
        else:
            if center_lanes>center_x_old:
                center_x = center_lanes/2
            elif center_lanes<center_x_old:
                center_x = (center_lanes+128)/2
            
    elif len(point_1) == 2:
        cv2.putText(img,'2 lanes detected',(0,120),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        center_x=(point_1[0]+point_1[1]+point_2[0]+point_2[1])/4
        # straight
        if abs(point_1[0]-point_2[0]) < 15 and abs(point_1[1]-point_2[1]) < 15 and junction_flag == 0:
            straight_count+=1
        if straight_count>100:
            straight_flag = 1
            
    elif len(point_1) == 0:
        cv2.putText(img,'nothing is detected',(5,120),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        center_x=center_x_old
        
    if straight_flag == 1:
        cv2.putText(img,'straight',(5,30),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        cv2.rectangle(img, (center_x_old-15,34), (center_x_old+15,54), color, thickness)

        # junction?
        img_roi = img_eroded[44:64,center_x_old-15:center_x_old+15]
        if np.sum(np.apply_along_axis(np.count_nonzero, 0, img_roi)) > 15 and junction_flag == 0:
            cv2.putText(img,'junction',(5,50),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)

            junction_flag = 1
            junction_frame = count
            straight_flag = 0
            straight_count = 0
            
    if (junction_flag==1 and len(point_1)==1) == False:      
        if center_x_old!=0 and abs(center_x-center_x_old)>tolerance:
            center_x=center_x_old 
    
    if count > junction_frame + 100:
        junction_flag = 0
    
    if junction_flag == 1:
        cv2.putText(img,'junc_flag',(5,70),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        
        if direction == "left":
            cv2.putText(img,'turn left',(60,10),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
        if direction == "right":
            cv2.putText(img,'turn right',(60,10),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
            
    center_x_old = center_x
    
    cv2.circle( img, (center_x, (min_y+max_y)/2), 4, color, -1, 8 )
    cv2.putText(img,'center',(center_x, (min_y+max_y)/2),cv2.FONT_HERSHEY_SIMPLEX, text_scale,color,thickness,cv2.LINE_AA)
    
    cv2.line(img, (0, min_y), (129, min_y), color, thickness)
    cv2.line(img, (0, max_y), (129, max_y), color, thickness)
    
    return center_x_old, straight_count, straight_flag, junction_flag, junction_frame
#    cv2.imwrite('/home/phamthinh/jupyter/media/fuck.jpg', img_roi)

In [41]:
count = 0
low_threshold = 150
high_threshold = 255
polygon = np.array([ [0,479], [740,479], [740,200] , [0,200] ], np.int32)
polygon2 = np.array([[362, 180], [0, 480], [1040, 480], [678, 180]], np.int32)
polygon2 = polygon2.reshape((-1,1,2))
polygon3 = np.array([[130, 270], [130, 480], [800, 480], [800, 270]])
polygon3 = polygon3.reshape((-1,1,2))
element = cv2.getStructuringElement(cv2.MORPH_RECT,(2*7+1,2*2+1))
element1 = cv2.getStructuringElement(cv2.MORPH_RECT,(2*3+1,2*1+1))
kernel = np.ones((2, 2), np.uint8)
center_x_old= 0
junction_flag = 0 
junction_frame = 0
# variable for lane_detection
direction = "right"
tolerance = 30
min_y=80
max_y=100
sum_required = 20
straight_count = 0
straight_flag = 0
#variable for thresholding
thresh = 220
maxValue = 255

cap = cv2.VideoCapture('/home/phamthinh/jupyter/media/clips/output13.avi')

frame_width = 128
frame_height = 128
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('media/output14.avi',cv2.VideoWriter_fourcc(*'40'), 30, (frame_width,frame_height)) 

IMAGE_H = 450
IMAGE_W = 640
src = np.float32([[362, 180], [0, 480], [678, 180], [1040, 480]])
#dst = np.float32([[200, 100], [200, IMAGE_H+30], [740, 100], [740, IMAGE_H+30]])
dst = np.float32([[200, 0], [200, IMAGE_H+30], [840, 0], [840, IMAGE_H+30]])

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        count+=1
        frame = cv2.flip(frame,1)
        frame = cv2.copyMakeBorder(frame, 0, 0, 200, 200, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        original_frame = frame;
        frame = transformImage(frame,src,dst)
        cv2.polylines(original_frame, [polygon2], True, (255,0,255), 10)
#        cv2.imshow("Original", original_frame)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #cv2.polylines(gray, [polygon3], True, (255,0,255), 10)
        gray = gray[0:480, 200:840]
        gray = cv2.resize(gray, (128, 128), interpolation=cv2.INTER_LINEAR)
#        cv2.imshow("Transformed", gray)
#        cv2.waitKey(1)
        # Set thamehold and maxValue
    
        #### added from video processing 

        # Basic threshold example
        th, img_thres = cv2.threshold(gray, thresh, maxValue, cv2.THRESH_BINARY);

        img_eroded = cv2.erode(img_thres, kernel, iterations=1)

        center_x_old, straight_count, straight_flag, junction_flag, junction_frame =center_path(gray, img_eroded,min_y,max_y, direction, center_x_old, tolerance, sum_required, straight_count, straight_flag, count, junction_flag, junction_frame)
#        out.write(gray)
        #### added from video processing
        cv2.imwrite('/home/phamthinh/jupyter/media/bird view frames/fr_g%d.jpg' %count, gray)
    else:
        break
        
cap.release()
#out.release()
 
# Closes all the frames
cv2.destroyAllWindows() 

In [3]:
test_img = cv2.imread('/media/azure/446455ed-0b48-42de-a754-fdd085a8d1791/github/CarND-Advanced-Lane-Lines-master/videos/frames_original/frame1.jpg')
test_img = cv2.copyMakeBorder(test_img, 0, 0, 200, 200, cv2.BORDER_CONSTANT, value=[0, 0, 0])
IMAGE_H = 450
#IMAGE_H = 875
IMAGE_W = 1040
src = np.float32([[362, 180], [0, 480], [678, 180], [1040, 480]])
dst = np.float32([[200, 0], [200, IMAGE_H+30], [740, 0], [740, IMAGE_H+30]])
#dst = np.float32([[200, 100], [200, IMAGE_H+100], [1240, 100], [1240, IMAGE_H+100]])
#test_img = transformImage(test_img,src,dst)
#polygon2 = np.array([[10,5],[20,30],[70,20],[50,10]], np.int32)
polygon2 = np.array([[362, 180], [0, 480], [1040, 480], [678, 180]], np.int32)
polygon2 = polygon2.reshape((-1,1,2))
cv2.polylines(test_img, [polygon2], True, (255,0,255), 10)

show_image(test_img)

In [45]:
file_object = open("/media/azure/446455ed-0b48-42de-a754-fdd085a8d1791/Dataset/Behavioral_cloning/data13/data13.txt", 'r')
print(file_object.read(5))

0 1.0
